In [ ]:
#nested cross validation to report inner_cv accuracy
outer_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)   
inner_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
inner_cv_scores=[]
outer_cv = 0
for train_index, test_index in outer_kf.split(X,y):
    
    X_outer_train, X_outer_test = X[train_index], X[test_index] 
    y_outer_train, y_outer_test = y[train_index], y[test_index]
    inner_cv = 0
    for train_index, test_index in inner_kf.split(X_outer_train,y_outer_train):  
        
        X_inner_train, X_inner_test = X_outer_train[train_index], X_outer_train[test_index] 
        y_inner_train, y_inner_test = y_outer_train[train_index], y_outer_train[test_index]
        
        #model fit & save weights & evaluate
        model.set_weights(W_Initial)            
        filepath = 'weights/' + model.name + str(outer_cv) + '_' + str(inner_cv) +'_best_pretrained_weights.npy'
        checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_weights_only=True,save_best_only=True, mode='auto', period=1)
        model.fit(x=X_inner_train, y=y_inner_train, validation_data=[X_inner_test, y_inner_test],
                  batch_size=nb_batch_size, epochs=nb_epoch, verbose=0, callbacks=[checkpoint]) 
                
        inner_cv_score = model.evaluate(x=X_inner_test, y=y_inner_test, batch_size=batch_size, verbose=1)
        inner_cv_scores.append(inner_cv_score)
        inner_cv+=1
    outer_cv+=1
    
print(inner_cv_scores)
print(np.mean(np.array(inner_cv_score), aixs=0))

In [ ]:
#10 repeats of outer_cv
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)   
nb_repeat 10
repeat_scores=[]
for i in range(nb_repeat):
    cv = 0
    cv_scores=[]
    for train_index, test_index in kf.split(X,y):

        X_train, X_test = X[train_index], X[test_index] 
        y_train, y_test = y[train_index], y[test_index]

        #model fit & save weights & evaluate
        model.set_weights(W_Initial)            
        filepath = 'weights/' + model.name + str(i) + '_' + str(cv) +'_best_pretrained_weights.npy'
        checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_weights_only=True,save_best_only=True, mode='auto', period=1)
        model.fit(x=X_inner_train, y=y_inner_train, validation_data=[X_inner_test, y_inner_test],
                  batch_size=nb_batch_size, epochs=nb_epoch, verbose=0, callbacks=[checkpoint])
        cv_score = model.evaluate(x=X_inner_test, y=y_inner_test, batch_size=batch_size, verbose=1)
        print(cv_score)
        cv_scores.append(cv_score)
        cv+=1
    repeat_scores.append(cv_scores)

In [ ]:
#nested_cv with 3 channels
outer_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)   
inner_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
inner_cv_scores=[]

outer_cv = 0
for train_index, test_index in outer_kf.split(X_1,y):
    
    X_1_outer_train, X_1_outer_test = X_1[train_index], X_1[test_index] 
    X_2_outer_train, X_2_outer_test = X_2[train_index], X_2[test_index]
    X_3_outer_train, X_3_outer_test = X_3[train_index], X_3[test_index]
    y_outer_train, y_outer_test = y[train_index], y[test_index]
    inner_cv = 0
    for train_index, test_index in inner_kf.split(X_1_outer_train,y_outer_train):  
        
        X_1_inner_train, X_1_inner_test = X_1_outer_train[train_index], X_1_outer_train[test_index] 
        X_2_inner_train, X_2_inner_test = X_2_outer_train[train_index], X_2_outer_train[test_index]
        X_3_inner_train, X_3_inner_test = X_3_outer_train[train_index], X_3_outer_train[test_index]
        y_inner_train, y_inner_test = y_outer_train[train_index], y_outer_train[test_index]
        Y_inner_train = np_utils.to_categorical(y_inner_train, nb_classes)
        Y_inner_test = np_utils.to_categorical(y_inner_test, nb_classes)
        
        #model fit & save weights & evaluate
        model.set_weights(W_Initial)            
        filepath = 'weights/nested_cv/' + model.name + str(outer_cv) + '_' + str(inner_cv) +'_best_pretrained_weights.npy'
        checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_weights_only=True,save_best_only=True, mode='auto', period=1)
        model.fit(x=[X_1_inner_train,X_2_inner_train,X_3_inner_train], y=Y_inner_train, validation_data=([X_1_inner_test,X_2_inner_test,X_3_inner_test],Y_inner_test),
                  batch_size=batch_size, epochs=nb_epoch, verbose=0, callbacks=[checkpoint]) 
                
        inner_cv_score = model.evaluate(x=[X_1_inner_test,X_2_inner_test,X_3_inner_test], y=Y_inner_test, batch_size=batch_size, verbose=1)
        inner_cv_scores.append(inner_cv_score)
        inner_cv+=1
    outer_cv+=1
    
print(inner_cv_scores)
print(np.mean(np.array(inner_cv_scores), aixs=0))